In [4]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, mean_absolute_error

data = pd.read_csv("./data/train_data.csv")
data = data.iloc[:, 1:]
data.head()

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,Action,time,event
0,0.62,0.03,0.10,0.28,0.43,0.67,0.38,0.12,0.15,0.88,0.72,0.89,0.43,0.36,0.43,0.66,0.16,0,0.47,1
1,0.12,0.87,0.43,0.28,0.02,0.23,0.82,0.14,0.81,0.68,0.53,0.24,0.05,0.01,0.96,0.98,0.34,0,1.39,1
2,0.37,0.06,0.47,0.86,0.86,0.33,0.01,0.90,0.79,0.43,0.64,0.84,0.05,0.56,0.01,0.77,0.62,0,3.61,1
3,0.57,0.39,0.65,0.76,0.83,0.92,0.57,0.60,0.34,0.35,0.44,0.93,0.44,0.65,0.61,0.25,0.96,0,1.02,1
4,0.82,0.00,0.26,0.27,0.82,0.68,0.43,0.92,0.81,0.26,0.66,0.04,0.10,0.93,0.82,0.37,0.09,1,2.07,1


In [5]:
data = data.drop(np.where(data.time == 0)[0].tolist())
data

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,Action,time,event
0,0.62,0.03,0.10,0.28,0.43,0.67,0.38,0.12,0.15,0.88,0.72,0.89,0.43,0.36,0.43,0.66,0.16,0,0.47,1
1,0.12,0.87,0.43,0.28,0.02,0.23,0.82,0.14,0.81,0.68,0.53,0.24,0.05,0.01,0.96,0.98,0.34,0,1.39,1
2,0.37,0.06,0.47,0.86,0.86,0.33,0.01,0.90,0.79,0.43,0.64,0.84,0.05,0.56,0.01,0.77,0.62,0,3.61,1
3,0.57,0.39,0.65,0.76,0.83,0.92,0.57,0.60,0.34,0.35,0.44,0.93,0.44,0.65,0.61,0.25,0.96,0,1.02,1
4,0.82,0.00,0.26,0.27,0.82,0.68,0.43,0.92,0.81,0.26,0.66,0.04,0.10,0.93,0.82,0.37,0.09,1,2.07,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661,0.50,0.99,0.61,0.88,0.68,0.34,0.96,1.00,0.19,0.80,0.14,0.29,0.41,0.40,0.73,0.76,0.01,1,4.77,1
662,0.62,0.19,0.51,0.30,0.12,0.10,0.41,0.25,0.17,0.17,0.73,0.20,0.18,0.10,0.19,0.39,0.17,1,0.03,1
663,0.84,0.30,0.99,0.55,0.83,0.98,0.17,0.33,0.83,0.45,0.68,0.51,0.57,0.46,0.71,0.29,0.37,0,3.60,1
664,0.94,0.83,0.37,0.69,0.92,0.62,0.39,0.41,0.20,0.74,0.60,0.46,0.21,0.06,0.23,0.93,0.26,0,6.52,1


In [6]:
X = data.iloc[:, :18]
y = data.iloc[:, 18]

X_train, X_val, y_train, y_val = train_test_split(X, np.log(y), test_size=0.2)

In [7]:
X_train.iloc[:, 17].shape

(525,)

In [8]:
y_train.shape

(525,)

In [9]:
for i in range(18):
    dataset = lgb.Dataset(X_train.iloc[:, i].to_numpy().reshape(-1, 1), label=y_train.to_numpy())
    
    params = {'learning_rate': 0.0001, 
          'max_depth': 16, 
          'boosting': 'gbdt', 
          'objective': 'regression', 
          'metric': 'mse', 
          'is_training_metric': True, 
          'num_leaves': 144, 
          'feature_fraction': 0.9, 
          'bagging_fraction': 0.7, 
          'bagging_freq': 5, 
          'seed':2018}
    
    regressor = lgb.train(params, dataset, 10000)
    pred = regressor.predict(X_val)
    error = mean_absolute_error(y_val, pred)
    
    print(f"{i} - {error.round(2)}")
    

0 - 1.44
1 - 1.42
2 - 1.45
3 - 1.41
4 - 1.45
5 - 1.43
6 - 1.46
7 - 1.42
8 - 1.41
9 - 1.45
10 - 1.44
11 - 1.42
12 - 1.4
13 - 1.43
14 - 1.44
15 - 1.45
16 - 1.42
17 - 1.41


In [10]:
dataset = lgb.Dataset(X_train.iloc[:, [5, 12]], label=y_train)

params = {'learning_rate': 0.00001, 
      'max_depth': 50, 
      'boosting': 'gbdt', 
      'objective': 'regression', 
      'metric': 'mse', 
      'is_training_metric': True, 
      'num_leaves': 250, 
      'feature_fraction': 0.9, 
      'bagging_fraction': 0.7, 
      'bagging_freq': 30, 
      'seed':2018}

regressor = lgb.train(params, dataset, 10000)
pred = regressor.predict(X_val)

In [11]:
from sklearn.metrics import mean_squared_error
import numpy as np

# np.sqrt(mean_squared_error(y_val, pred))
mean_absolute_error(y_val, pred)

1.4147263202188147

In [12]:
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(X_train, y_train)
pred = regressor.predict(X_val)

np.sqrt(mean_squared_error(y_val, pred))

0.8319939029292194

In [68]:
test_data = pd.read_csv('./data/testX.csv')
test_data = test_data.iloc[:, 1:]

pred = clf.predict(test_data.to_numpy())

In [69]:
result = pd.DataFrame(zip(list(range(287)), map(int, pred.tolist())), columns=['Title', 'action'])
result.to_csv('./result/tabnet_result.csv', index=False)
result

,Title,action
0,0,0
1,1,0
2,2,0
3,3,0
4,4,1
...,...,...
281,281,1
282,282,0
283,283,0
284,284,0
